

# 이상형 월드컵을 통한 서울시 자취방 추천

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import os
from tkinter import *
import smtplib
from email.mime.text import MIMEText
import tkinter.ttk as ttk
import requests
from urllib.parse import urlparse
from tqdm import tqdm
from scipy import spatial

import warnings
warnings.filterwarnings(action='ignore')

### 크롤링 코드

In [2]:
class croll:
    def __init__(self):
        self.rent = ""
        self.district = ""
        self.neighborhood = ""
        self.email = ""
    
    def input_data(self,rent,district,neighborhood,email):
        self.rent = rent
        self.district = district
        self.neighborhood = neighborhood
        self.email = email
        
    def crolling(self):
        # 브라우저 생성
        # browser = webdriver.Chrome('C:/chromedriver.exe')
        browser = webdriver.Chrome('C:/Users/user/Desktop/이상형월드컵/chromedriver.exe')

        # 웹사이트 열기
        browser.get("https://land.naver.com/")
        # 원룸.투룸 선택 하기
        browser.find_element_by_link_text('원룸·투룸').click()
        browser.find_element_by_link_text(self.rent).click()
        browser.find_element_by_link_text('리스트로 보기').click()
        browser.find_element_by_link_text('서울시').click()
        browser.implicitly_wait(1) # 클릭 대기
        browser.find_element_by_link_text(self.district).click()
        browser.implicitly_wait(1) # 클릭 대기
        browser.find_element_by_partial_link_text(self.neighborhood+" ").click()
        browser.find_element_by_link_text("확인 매물 보기").click()
        # 매물 가져오기 

        # 스크롤 전 높이
        itemlist = browser.find_element_by_xpath('//*[@id="listContents1"]/div')
        before_h=browser.execute_script("return arguments[0].scrollHeight;",itemlist)

        # 무한 스크롤(scroll in scroll)
        while True:
            #맨 아래로 스크롤을 내린다.
            browser.execute_script("arguments[0].scrollBy(0,arguments[0].scrollHeight)", itemlist)

            # 스크롤 사이 페이지 로딩 시간
            time.sleep(1)

            # 스크롤 후 높이
            after_h=browser.execute_script("return arguments[0].scrollHeight;",itemlist)

            if after_h == before_h:
                break
            before_h = after_h

        # 파일 생성
        f = open(r"C:\Users\user\Desktop\이상형월드컵\data_"+self.district+"_"+self.neighborhood+".csv",'w',encoding='utf-8-sig',newline='')

        csvWriter = csv.writer(f)
        csvWriter.writerow(["방이름","보증금","월세","전세","방 수","욕실 수","주소","평 수","관리비","주차 여부","방향","층","총 층","건축물 용도","전세여부","월세여부","부동산 이름","부동산 전화번호","복층 여부","사용승인일","내부시설","기타시설"])


        # 매물 정보 div
        items = browser.find_elements_by_css_selector('div.item')
        

        for item in items:
            # 피터팬
            try:
        #         item.find_element_by_css_selector("a.label").click()
                item.find_element_by_xpath('//*[@id="listContents1"]/div/div/div[1]/div[2]/div/div[2]/a').click()
                browser.implicitly_wait(1)


                if item == items[0]:
                    item.find_element_by_xpath('//*[@id="tabArea"]/button').click()

                dict = {}
                try:
                    name = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[2]/h4').text
                except:
                    name = item.find_element_by_css_selector("span.text").text

                try:
                    price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[3]/span[2]').text

                except:
                    price = "none"

                if(price=="none"):
                    try:
                        price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[2]').text
                    except:
                        price = "none"

                if(price == 'none'):
                    continue

                find_slash = price.find('/')
                if (self.rent == "월세"):
                    bo = price[:find_slash]
                    monthly_price = price[find_slash+1:]
                    yearly_price = 0
                else:
                    bo = 0
                    monthly_price = 0
                    yearly_price = price

                info_list = item.find_elements_by_xpath('//*[@id="detailContents1"]/div[1]/table/tbody/tr')

                for i in info_list:
                    key_list = i.find_elements_by_class_name('table_th')
                    value_list = i.find_elements_by_class_name('table_td')
                    for num in range(len(key_list)):
                        dict[key_list[num].text] = value_list[num].text
                # 방수/욕실수 
                try:
                    room = dict['방수/욕실수']
                    find_slash = room.find('/')
                    room_num = room[find_slash-1:find_slash]
                    bath_num = room[find_slash+1:find_slash+2]
                except:
                    room = "none"
                    room_num = 0
                    bath_num = 0

                # 주소    
                try:
                    address = dict['소재지']
                except:
                    address = "none"

                if(address == "none"):
                    continue

                # 면적    
                try:
                    size = dict['공급/전용면적']
                except:
                    size = "none"

                if(size=='none'):
                    continue

                # 관리비
                try:
                    administrative_expenses = dict['월관리비']
                except:
                    administrative_expenses = "none"

                # 주차공간
                try:
                    parking = dict['주차가능여부']
                except:
                    parking = "none"

                # 방향
                try:
                    direction = dict['방향']
                    find_garo = dict['방향'].find("(")
                    if(find_garo != -1):
                        direction = direction[:find_garo]
                except:
                    direction = "none"

                # 층수
                try: 
                    floor = dict['해당층/총층']
                    find_slash = floor.find('/')
                    floor_num = floor[:find_slash]
                    all_num = floor[find_slash+1:find_slash+2]
                except:
                    floor = "none"
                    floor_num = "0"
                    all_num = "0"

                # 건축물 용도    
                try:
                    for_what = dict['건축물 용도']
                except:
                    for_what = "none"

                # 부동산 정보
                try:
                    real_estate = dict['중개사']
                    if(real_estate.find("톡톡")!= -1):
                        real_estate_name = real_estate[:real_estate.find("톡톡")]
                        real_estate_name.strip('\n')
                    elif(real_estate.find("길찾기")!=-1):
                        real_estate_name = real_estate[:real_estate.find("길찾기")]
                        real_estate_name.strip('\n')
                    else:
                        real_estate_name = real_estate[:real_estate.find("대표")]
                        real_estate_name.strip('\n')

                    if(real_estate.find("최근") != -1):
                        real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("최근")-1]
                    elif(real_estate.find("매매")!= -1):
                        real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("매매")-1]
                    else:
                        real_estate_num = real_estate[real_estate.find("전화")+2:]
                except:
                    real_estate_name = "none"
                    real_estate_number = "none"

                # 복층 여부
                try:
                    is_single = dict['복층여부']
                except:
                    is_single = 'none'

                # 사용 승인일
                try:
                    date = dict['사용승인일']
                except:
                    date = 'none'

                # 월세, 전세 
                if (self.rent == "월세"):
                    monthly = 1
                    yearly = 0
                else:
                    monthly = 0
                    yearly = 1

                # 내부시설
                try:
                    option = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[2]')
                    option_list = option.find_elements_by_css_selector('div.option_item')
                    option_data = ""
                    for option in option_list:
                        option_data += option.text+", "
                except:
                    option_data = "none"

                # 기타시설
                try:
                    etc = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[4]')
                    etc_list = etc.find_elements_by_css_selector('div.option_item')
                    etc_data = ""
                    
                    for etc in etc_list:
                        etc_data += etc.text + ", "
                        
                except:
                    etc_data = "none"

                csvWriter.writerow([name,bo,monthly_price,yearly_price,room_num,bath_num,address,size,administrative_expenses,parking,direction,floor_num,all_num,for_what,yearly,monthly,real_estate_name,real_estate_num,is_single,date,option_data,etc_data])


            except:
                item.find_element_by_css_selector("a.item_link").click()
                browser.implicitly_wait(1)

                if item == items[0]:
                    item.find_element_by_xpath('//*[@id="tabArea"]/button').click()
                dict = {}

                try:
                    name = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[2]/h4').text
                except:
                    name = item.find_element_by_css_selector("span.text").text

                try:
                    price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[3]/span[2]').text
                except:
                    price = "none"

                if(price=="none"):
                    try:
                        price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[2]').text
                    except:
                        price = "none"

                if(price == 'none'):
                    continue

                find_slash = price.find('/')
                if (self.rent == "월세"):
                    bo = price[:find_slash]
                    monthly_price = price[find_slash+1:]
                    yearly_price = 0
                else:
                    bo = 0
                    monthly_price = 0
                    yearly_price = price

                info_list = item.find_elements_by_xpath('//*[@id="detailContents1"]/div[1]/table/tbody/tr')

                for i in info_list:
                    key_list = i.find_elements_by_class_name('table_th')
                    value_list = i.find_elements_by_class_name('table_td')
                    for num in range(len(key_list)):
                        dict[key_list[num].text] = value_list[num].text
                # 방수/욕실수 
                try:
                    room = dict['방수/욕실수']
                    find_slash = room.find('/')
                    room_num = room[find_slash-1:find_slash]
                    bath_num = room[find_slash+1:find_slash+2]
                except:
                    room = "none"
                    room_num = 0
                    bath_num = 0

                # 주소    
                try:
                    address = dict['소재지']
                except:
                    address = "none"

                if(address == "none"):
                    continue

                # 면적    
                try:
                    size = dict['공급/전용면적']
                except:
                    size = "none"

                if (size=="none"):
                    continue

                # 관리비
                try:
                    administrative_expenses = dict['월관리비']
                except:
                    administrative_expenses = "none"

                # 주차공간
                try:
                    parking = dict['주차가능여부']
                except:
                    parking = "none"

                # 방향
                try:
                    direction = dict['방향']
                    find_garo = dict['방향'].find("(")
                    if(find_garo != -1):
                        direction = direction[:find_garo]
                except:
                    direction = "none"

                # 층수
                try: 
                    floor = dict['해당층/총층']
                    find_slash = floor.find('/')
                    floor_num = floor[:find_slash]
                    all_num = floor[find_slash+1:find_slash+2]
                except:
                    floor = "none"
                    floor_num = "0"
                    all_num = "0"

                # 건축물 용도    
                try:
                    for_what = dict['건축물 용도']
                except:
                    for_what = "none"

                # 부동산 정보
                try:
                    real_estate = dict['중개사']
                    if(real_estate.find("톡톡")!= -1):
                        real_estate_name = real_estate[:real_estate.find("톡톡")]
                        real_estate_name.strip('\n')
                    elif(real_estate.find("길찾기")!=-1):
                        real_estate_name = real_estate[:real_estate.find("길찾기")]
                        real_estate_name.strip('\n')
                    else:
                        real_estate_name = real_estate[:real_estate.find("대표")]
                        real_estate_name.strip('\n')

                    if(real_estate.find("최근") != -1):
                        real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("최근")-1]
                    elif(real_estate.find("매매")!= -1):
                        real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("매매")-1]
                    else:
                        real_estate_num = real_estate[real_estate.find("전화")+2:]
                except:
                    real_estate_name = "none"
                    real_estate_number = "none"

                # 복층 여부
                try:
                    is_single = dict['복층여부']
                except:
                    is_single = 'none'

                # 사용 승인일
                try:
                    date = dict['사용승인일']
                except:
                    date = 'none'

                # 월세, 전세 
                if (self.rent == "월세"):
                    monthly = 1
                    yearly = 0
                else:
                    monthly = 0
                    yearly = 1

                
                # 내부시설
                try:
                    option = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[2]')
                    option_list = option.find_elements_by_css_selector('div.option_item')
                    option_data = ""
                    for option in option_list:
                        option_data += option.text+", "
                except:
                    option_data = "none"

                # 기타시설
                try:
                    etc = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[4]')
                    etc_list = etc.find_elements_by_css_selector('div.option_item')
                    etc_data = ""

                    for etc in etc_list:
                        etc_data += etc.text + ", "
                       
                except:
                    etc_data = "none"

                csvWriter.writerow([name,bo,monthly_price,yearly_price,room_num,bath_num,address,size,administrative_expenses,parking,direction,floor_num,all_num,for_what,yearly,monthly,real_estate_name,real_estate_num,is_single,date,option_data,etc_data])

        # 파일 닫기
        f.close()


## 전처리 코드

In [3]:
def convert_str_to_int(x):
    convert = int(x)
    return convert
    
def convert_floor(x):
    if (x=='단층'or x=='none'):
        return 0
    elif (x=='복층'):
        return 1
    
def convert_mil(x):
    mil = x.find('억')
    if(mil!=-1):
        return x[:mil]+x[mil+2:]
    else:
        return x
        
def convert_money(x):
    rest = x.find(',')
    if(rest!=-1):
        str_money = x[:rest]+x[rest+1:]
        return convert_str_to_int(str_money)
    else:
        return convert_str_to_int(x)
        
def convert_size(x):
    find_slash = x.find('/')
    first_pyung = x.find('평')
    if(first_pyung != -1):
        second_pyung = x.find('평',first_pyung+1)
        size = float(x[find_slash+1:second_pyung])
    else:
        find_a = x.find('(')
        size = float(x[find_slash+1:find_a-1])
        size = size/3.3058
    return size

def transform_address(address):
    rest_api_key = '97527267926acc1dce473f884f0ec30a'
    url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":f"KakaoAK {rest_api_key}"})
    json_obj = result.json()
    # 값이 NoneType 일 경우 오류 방지
    try:
        document = json_obj['documents'][0]
        val = [document['x'], document['y']]
    except:
        val = [0, 0]
    
    return val

def category_count(address, place):
    rest_api_key = '97527267926acc1dce473f884f0ec30a'
    x, y = address[0], address[1]
    if x==0 and y==0:
        return np.nan
    url = "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=" + place[0] + "&x=" + str(x) + "&y=" + str(y) + "&radius=" + str(place[1])
    result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":f"KakaoAK {rest_api_key}"})
    json_obj = result.json()
     # 값이 NoneType 일 경우 오류 방지
    try:
        min = place[1]
        for document in json_obj['documents']:
            if int(document['distance']) < min:
                min = int(document['distance'])
    except:
        min = np.nan
    return min


In [4]:
class preprocess:
    
    def __init__(self,district,neighborhood):
        self.data = "data_" + district +"_"+neighborhood
        self.df = pd.read_csv(self.data+".csv",encoding = '')
        
    

    def process(self):
        self.df['방 수'] = self.df['방 수'].apply(convert_str_to_int)
        self.df['복층 여부'] = self.df['복층 여부'].apply(convert_floor)
        self.df['평 수'] = self.df['평 수'].apply(convert_size)
        
        for i in range(len(self.df)):
            if self.df.loc[i, '욕실 수'] == '-':
                self.df.loc[i, '욕실 수'] = 1
        
        self.df['욕실 수'] = self.df['욕실 수'].astype('int64')
        
        for i in range(len(self.df)):
            if self.df.loc[i, '사용승인일'] == "none" or self.df.loc[i,'사용승인일']=="-":
                self.df.loc[i, '사용승인일'] = "0"
        
        for i in range(len(self.df)):
            self.df.loc[i, '사용승인일'] = self.df.loc[i, '사용승인일'][:4]

        self.df['사용승인일'] = self.df['사용승인일'].astype('int64')
        
        date_sum = 0
        for i in range(len(self.df)):
            if self.df.loc[i, '사용승인일'] != 0:
                date_sum += self.df.loc[i, '사용승인일']

        for i in range(len(self.df)):
            if self.df.loc[i, '사용승인일'] == 0:
                self.df.loc[i, '사용승인일'] = math.trunc(date_sum / len(self.df))
                
        for i in range(len(self.df)):
            if self.df.loc[i, '관리비'] == "none":
                self.df.loc[i, '관리비'] = "0"
            self.df.loc[i, '관리비'] = self.df.loc[i, '관리비'].replace("만원", "")
            self.df.loc[i, '관리비'] = self.df.loc[i, '관리비'].replace("원", "")

        self.df['관리비'] = self.df['관리비'].astype('float64')
        
        cost_sum = 0
        for i in range(len(self.df)):
            if self.df.loc[i, '관리비'] != 0:
                cost_sum += self.df.loc[i, '관리비']

        for i in range(len(self.df)):
            if self.df.loc[i, '관리비'] == 0:
                self.df.loc[i, '관리비'] = math.trunc(cost_sum / len(self.df))
                
        self.df["월세"] = self.df["월세"].apply(convert_str_to_int)
        
        if(str(type(self.df['보증금'].iloc[0]))!= "<class 'numpy.int64'>"):
            self.df['보증금'] = self.df['보증금'].apply(convert_mil)
            self.df['보증금'] = self.df['보증금'].apply(convert_money)
            
        if(str(type(self.df['전세'].iloc[0]))!= "<class 'numpy.int64'>"):
            self.df['전세'] = self.df['전세'].apply(convert_mil)
            self.df['전세'] = self.df['전세'].apply(convert_money)
            self.df['전세']
            
        for i in range(len(self.df)):
            floor = self.df.loc[i, '층']
            total = int(self.df.loc[i, '총 층'])
            if floor=='고':
                self.df.loc[i, '층'] = total
            elif floor=='중':
                self.df.loc[i, '층'] = total//3*2
            elif floor=='저':
                self.df.loc[i, '층'] = total//3
            elif floor=='B1':
                self.df.loc[i, '층'] = -1
            else:
                self.df.loc[i, '층'] = int(floor)
                
        self.df['총 층'] = self.df['총 층'].apply(convert_str_to_int)
        
        for i in range(len(self.df)):
            if self.df.loc[i, '주차 여부'] == '불가능':
                self.df.loc[i, '주차 여부'] = '0'
            elif self.df.loc[i, '주차 여부'] == '가능':
                self.df.loc[i, '주차 여부'] = 1
            elif self.df.loc[i, '주차 여부'] == 'none':
                self.df.loc[i, '주차 여부'] = '0'
            
        # 방향
        for i in range(len(self.df)):
            if self.df.loc[i, '방향'] == 'none':
                self.df.drop(index = [i], inplace=True)

        east = []
        for i in range(len(self.df)):
            if '동향' in self.df.loc[i, '방향'] and '남동향' not in self.df.loc[i, '방향'] and '북동향' not in self.df.loc[i, '방향']:
                east.append(1)
            elif '남동향' in self.df.loc[i, '방향'] or '북동향' in self.df.loc[i, '방향']:
                east.append(0.5)
            else:
                east.append(0)
        self.df['방향_동'] = east

        west = []
        for i in range(len(self.df)):
            if '서향' in self.df.loc[i, '방향'] and '남서향' not in self.df.loc[i, '방향'] and '북서향' not in self.df.loc[i, '방향']:
                west.append(1)
            elif '남서향' in self.df.loc[i, '방향'] or '북서향' in self.df.loc[i, '방향']:
                west.append(0.5)
            else:
                west.append(0)    
        self.df['방향_서'] = west

        south = []
        for i in range(len(self.df)):
            if '남향' in self.df.loc[i, '방향'] and '남동향' not in self.df.loc[i, '방향'] and '남서향' not in self.df.loc[i, '방향']:
                south.append(1)
            elif '남동향' in self.df.loc[i, '방향'] or '남서향' in self.df.loc[i, '방향']:
                south.append(0.5)
            else:
                south.append(0)
        self.df['방향_남'] = south

        north = []
        for i in range(len(self.df)):
            if '북향' in self.df.loc[i, '방향'] and '북동향' not in self.df.loc[i, '방향'] and '북서향' not in self.df.loc[i, '방향']:
                north.append(1)
            elif '북동향' in self.df.loc[i, '방향'] or '북서향' in self.df.loc[i, '방향']:
                north.append(0.5)
            else:
                north.append(0)
        self.df['방향_북'] = north

        self.df.drop(columns = '방향', inplace=True)
        
        for i in range(len(self.df)):
            if self.df.loc[i, '건축물 용도'] == 'none':
                self.df.drop(index = [i], inplace=True)

        self.df = pd.get_dummies(self.df, prefix = '건축물 용도', columns = ['건축물 용도'])
        
        self.df['주소좌표'] = self.df['주소'].apply(transform_address)
        
        places = [['MT1', 1500, '대형마트'], ['CS2', 400, '편의점'], ['SW8', 1200, '지하철역']]
        for place in places:
            colname = place[2]
            self.df[colname] = self.df['주소좌표'].apply(category_count, args=(place,))
        
        self.df.to_csv("C:/Users/user/Desktop/이상형월드컵/"+self.data+"_preprocessed.csv",encoding="utf-8-sig",index=False)

## 이상형월드컵 코드

In [5]:
class WorldCup:
    filenamelist=os.listdir('.\\image')
    
    rent = "" # 전/월세
    district = "" # 구
    neighborhood = "" # 동
    email = "" # 이메일

    # dictionary
    c1 = -1 # 우승
    c2 = [] # 결승
    c4 = [] # 4강
    c8 = [] # 8강
    c16 = [] # 16강
    # 이미지
    im = [] #round에 따른 초기화 값 저장
    im1 = -1
    im2 = []
    im4 = []
    im8 = []
    im16 = []
    # 방 조건(출력용)
    rc1 = -1
    rc2 = []
    rc4 = []
    rc8 = []
    rc16 = []
    
    csm12 = [] #패자부활전 - dictionary
    csm1 = dict() #패자부활전 우승 - dictionary
    win_room = dict() # 우승한 방
    csm12_condition = [] # 패자부활전 (조건)
    csm1_condition = [] # 패자부활전 우승 (조건) (출력용)
    
    c_cup = {1:c1, 2:c2, 4:c4, 8:c8, 16:c16} # 방 조건 dictionary (수치화)
    im_cup = {1:im1, 2:im2, 4:im4, 8:im8, 16:im16} # 방 이미지
    rc_cup = {1:rc1, 2:rc2, 4:rc4, 8:rc8, 16:rc16} # 방 조건 문구 (출력용)
    
    result = []
    fname = [] # 파일명
    
    round = 16 # 강 수
    i = 0
    
    ### 방 조건 dictionary ###
    room1 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1, '평수': 6,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 262, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room2 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 2, '욕실 수': 1, '평수': 11.5,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 492, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room3 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 2, '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': 2, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room4 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1, '평수': 11.5,
            '관리비': 8, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room5 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 2, '욕실 수': 1, '평수': 8.62,
            '관리비': 8, '주차여부': 0.45, '층': 6, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room6 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1, '욕실 수': 1, '평수': 8.62,
            '관리비': 4, '주차여부': 0.45, '층': 1, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room7 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1, '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': -1, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 262, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room8 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1, '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 2016,
            '대형마트': 447, '편의점': 83, '지하철': 492, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room9 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1, '평수': 8.62,
            '관리비': 5.5, '주차여부': 1, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 600, '편의점': 83, '지하철': 411, '방향_동': 1, '방향_서': 1, '방향_남': 0, '방향_북': 0}
    room10 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1, '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.1, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 292, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 1, '방향_북': 0}
    room11 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1,  '평수': 6,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 1, '사용승인일': 1984,
            '대형마트': 447, '편의점': 52, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room12 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1,  '평수': 11.5,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.1, '사용승인일': 1970,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room13 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1, '욕실 수': 1,  '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 2016,
            '대형마트': 447, '편의점': 105, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room14 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1,  '평수': 8.62,
            '관리비': 4, '주차여부': 0.45, '층': 6, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 1, '방향_서': 1, '방향_남': 0, '방향_북': 0}
    room15 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 1,  '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': 2, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1970,
            '대형마트': 600, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    room16 = {'보증금': 'none', '월세': 'none', '전세': 'none', '방 수': 1.38, '욕실 수': 2,  '평수': 8.62,
            '관리비': 5.5, '주차여부': 1, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 105, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    
    room_final = {'보증금': 0, '월세': 0, '전세': 0, '방 수': 1.38, '욕실 수': 1, '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
    
    ### 방 조건 ###
    room_con1 = "평수가 작지만 집 앞에 편의점이 있고 역세권"
    room_con2 = "방이 2개이고 넓지만 근처에 지하철역이 없음"
    room_con3 = "화장실이 2개이고 중간층이지만 북향인 방"
    room_con4 = "복층이고 평수가 크지만 관리비가 비싼 방"
    room_con5 = "관리비가 비싸지만 방이 2개이고 고층"
    room_con6 = "관리비가 싸지만 방이 1개이고 1층"
    room_con7 = "가격이 저렴하고 역세권이지만 반지하"
    room_con8 = "신축이고 가격이 저렴하지만 역세권이 아님"
    room_con9 = "주차가능, 남˙북향이 아니고 대형마트가 멀음"
    room_con10 = "주차불가능, 남향이고 대형마트가 가까움"
    room_con11 = "복층이지만 집 앞에 편의점이 있는 북향인 방"
    room_con12 = "복층이 아니면서 평수가 크고 오래된 방"
    room_con13 = "신축이고 남향이지만 방이 1개이고 편의점이 없음"
    room_con14 = "남향이 아니고 관리비가 싸며 고층인 방"
    room_con15 = "중간층이고 대형마트가 가깝지만 오래된 방"
    room_con16 = "화장실이 2개이고 주차가 가능, 편의점이 없음"
    
    room_list = [room1,room2,room3,room4,room5,room6,room7,room8,room9,room10,room11,room12,room13,room14,room15,room16]
    room_condition_list = [room_con1,room_con2,room_con3,room_con4,room_con5,room_con6,room_con7,room_con8,
                     room_con9,room_con10,room_con11,room_con12,room_con13,room_con14,room_con15,room_con16]
    
    def data_setting(self): #변수 초기화
        self.c1=-1
        self.c2.clear()
        self.c4.clear()
        self.c8.clear()
        self.c16.clear()
        self.im.clear()
        self.im1=-1
        self.im2.clear()
        self.im4.clear()
        self.im8.clear()
        self.im16.clear()
        self.rc1=-1
        self.rc2.clear()
        self.rc4.clear()
        self.rc8.clear()
        self.rc16.clear()
        self.csm12.clear()
        self.csm1.clear()
        self.csm1_condition.clear()
        self.csm12_condition.clear()
        self.fname.clear()
        self.c_cup={1:self.c1, 2:self.c2, 4:self.c4, 8:self.c8, 16:self.c16}
        self.im_cup = {1:self.im1, 2:self.im2, 4:self.im4, 8:self.im8, 16:self.im16}
        self.rc_cup = {1:self.rc1, 2:self.rc2, 4:self.rc4, 8:self.rc8, 16:self.rc16}
        self.rent = ""
        self.monthly_price = 0 
        self.yearly_price = 0 
        self.deposit = 0 
        self.district = "" 
        self.neighborhood = "" 
        self.email = ""
        
    def get_rent(self):
        return self.rent
    
    def get_district(self):
        return self.district
    
    def get_neighborhood(self):
        return self.neighborhood
    
    def get_email(self):
        return self.email
    
    # 최종방
    def get_room(self):
        return self.room_final
    
    # 패자부활방
    def get_lose_room(self):
        return self.csm1
    
    # 우승방 
    def get_win_room(self):
        return self.win_room
    
    def load_file(self): # 파일명 리스트에 추가
        for filenames in self.filenamelist:
            self.fname.append(filenames)
            
    def show_room(self, round, i):  # 방 조건 (dictionary) 출력
        for key, value in list(self.c_cup[round][i].items()):
            if value == 'none':
                del self.c_cup[round][i][key]
        return self.c_cup[round][i]
    
    def show_room_condition(self, round, i): # 방 특징 출력
        if round == 1:
            return self.rc_cup[round]
        else:
            return self.rc_cup[round][i]
    
    def victory(self): # 우승
        window.geometry("1040x616")
        window.configure(bg = "#ffffff")
        canvas = Canvas(
            window,
            bg = "#ffffff",
            height = 616,
            width = 1040,
            bd = 0,
            highlightthickness = 0,
            relief = "ridge")
        canvas.place(x = 0, y = 0)
        
        win_room_condition = Label(window, text = "{}".format(self.show_room_condition(self.round, self.i)),
                            background = "white", font = ("Times", 16, "bold"))
        win_room_condition.place(x = 276, y = 514, width = 500, height = 57)
        
        winner_image = PhotoImage(file = f'.\\image\\' + self.fname[self.im_cup[self.round]])
        winner = Label(window, image = winner_image)
        winner.place(x = 270, y = 83, width = 500, height = 400)
        
        img1 = PhotoImage(file = f"csm_btn.png")
        b1 = Button(
            image = img1,
            borderwidth = 0,
            highlightthickness = 0,
            command = self.consolation_match,
            relief = "flat")

        b1.place(
            x = 774, y = 483,
            width = 240,
            height = 102)

        background_img = PhotoImage(file = f"end_background.png")
        background = canvas.create_image(
            508.0, 23.5,
            image=background_img)

        window.resizable(False, False)
        window.mainloop()
    
    def consolation_match(self): #패자부활전
        window.geometry("1040x616")
        window.configure(bg = "#ffffff")
        canvas = Canvas(
            window,
            bg = "#ffffff",
            height = 616,
            width = 1040,
            bd = 0,
            highlightthickness = 0,
            relief = "ridge")
        canvas.place(x = 0, y = 0)
        
        values = self.csm12_condition
        combobox = ttk.Combobox(window, width = 40, height = 12, values = values, state = "readonly")
        combobox.pack()
        combobox.set("추천조건에 넣고 싶은 매물")
        combobox.place(
            x = 360, y = 170)
        
        def decision():
            index = 0
            for i in range(12):
                if self.csm12_condition[i] == combobox.get():
                    index = i
            self.csm1 = self.csm12[index]
            
            self.room_final["월세"] = self.room_final["월세"]*(0.8) + self.csm1["월세"]*(0.2)
            self.room_final["보증금"] = self.room_final["보증금"]*(0.8) + self.csm1["보증금"]*(0.2)
            self.room_final["전세"] = self.room_final["전세"]*(0.8) + self.csm1["전세"]*(0.2)
            self.room_final["방 수"] = self.room_final["방 수"]*(0.8) + self.csm1["방 수"]*(0.2)
            self.room_final["욕실 수"] = self.room_final["욕실 수"]*(0.8) + self.csm1["욕실 수"]*(0.2)
            self.room_final["평수"] = self.room_final["평수"]*(0.8) + self.csm1["평수"]*(0.2)
            self.room_final["관리비"] = self.room_final["관리비"]*(0.8) + self.csm1["관리비"]*(0.2)
            self.room_final["주차여부"] = self.room_final["주차여부"]*(0.8) + self.csm1["주차여부"]*(0.2)
            self.room_final["층"] = self.room_final["층"]*(0.8) + self.csm1["층"]*(0.2)
            self.room_final["총 층"] = self.room_final["총 층"]*(0.8) + self.csm1["총 층"]*(0.2)
            self.room_final["복층 여부"] = self.room_final["복층 여부"]*(0.8) + self.csm1["복층 여부"]*(0.2)
            self.room_final["사용승인일"] = self.room_final["사용승인일"]*(0.8) + self.csm1["사용승인일"]*(0.2)
            self.room_final["대형마트"] = self.room_final["대형마트"]*(0.8) + self.csm1["대형마트"]*(0.2)
            self.room_final["편의점"] = self.room_final["편의점"]*(0.8) + self.csm1["편의점"]*(0.2)
            self.room_final["지하철"] = self.room_final["지하철"]*(0.8) + self.csm1["지하철"]*(0.2)
            self.room_final["방향_동"] = self.room_final["방향_동"]*(0.8) + self.csm1["방향_동"]*(0.2)
            self.room_final["방향_서"] = self.room_final["방향_서"]*(0.8) + self.csm1["방향_서"]*(0.2)
            self.room_final["방향_남"] = self.room_final["방향_남"]*(0.8) + self.csm1["방향_남"]*(0.2)
            self.room_final["방향_북"] = self.room_final["방향_북"]*(0.8) + self.csm1["방향_북"]*(0.2)
        
        btn = Button(window, text = "조건 추가", command = decision)
        btn.place(
            x = 470, y = 200,)
    
        img1 = PhotoImage(file = f"exit.png")
        b1 = Button(
            image = img1,
            borderwidth = 0,
            highlightthickness = 0,
            command = btn_exit,
            relief = "flat")

        b1.place(
            x = 774, y = 483,
            width = 240,
            height = 102)

        background_img = PhotoImage(file = f"csm_background.png")
        background = canvas.create_image(
            508.0, 43.5,
            image=background_img)
        

        window.resizable(False, False)
        window.mainloop()
        
    def start_game(self):
        window.geometry("1040x616")
        window.configure(bg = "#ffffff")
        canvas = Canvas(
            window,
            bg = "#ffffff",
            height = 616,
            width = 1040,
            bd = 0,
            highlightthickness = 0,
            relief = "ridge")
        canvas.place(x = 0, y = 0)
        
        self.rent = rent.get()
        self.deposit = deposit.get()
        self.district = district.get()
        self.neighborhood = neighborhood.get()
        self.email = email.get()
        
        if self.rent == "월세":
            self.monthly_price = rent_fee.get()
            
            for i in range(len(self.room_list)):
                self.room_list[i]["전세"] = 0
                self.room_list[i]["보증금"] = self.deposit
                self.room_list[i]["월세"] = self.monthly_price
                if i == 7 and 8:
                    self.room_list[i]["보증금"] = self.deposit - 200
                    self.room_list[i]["월세"] = self.monthly_price - 15
                    self.room_list[i]["전세"] = 0
                
        else:
            self.yearly_price = rent_fee.get()
            for i in range(len(self.room_list)):
                self.room_list[i]["보증금"] = self.deposit
                self.room_list[i]["전세"] = self.yearly_price
                self.room_list[i]["월세"] = 0
                if i == 7 and 8:
                    self.room_list[i]["보증금"] = self.deposit - 200
                    self.room_list[i]["전세"] = self.yearly_price - 500
                    self.room_list[i]["월세"] = 0
        
#         print("rent = {}, district = {}, neighborhood = {}, email = {}".format(self.rent, self.district,
#                                                                               self.neighborhood, self.email))
        
        if int(self.round) == 16: # 첫 라운드만 세팅
            self.data_setting()
            self.load_file()
            for rounds in range(self.round):
                self.im.append(rounds)
                
            self.c_cup[self.round] = self.room_list
            self.im_cup[self.round] = self.im
            self.rc_cup[self.round] = self.room_condition_list
        
        
            
        label = Label(window, text = "{}강 {}/{}".format(int(self.round), int((self.i+2)/2), int(self.round/2)),
                        background = "white", font = ("Times", 32))
        label.place(x = 0, y = 8, width = 200, height = 65)

        if self.round == 2:
            label = Label(window, text = "결승 {}/{}".format(int((self.i+2)/2), int(self.round/2)),
                        background = "white", font = ("Times", 32))
            label.place(x = 0, y = 8, width = 200, height = 65)
        room1_condition = Label(window, text = "{}".format(self.show_room_condition(self.round, self.i)),
                        background = "white", font = ("Times", 16, "bold"))
        room1_condition.place(x = 10, y = 538, width = 500, height = 57)

        
        room2_condition = Label(window, text = "{}".format(self.show_room_condition(self.round, self.i+1)),
                        background = "white", font = ("Times", 16, "bold"))
        room2_condition.place(x = 520, y = 538, width = 500, height = 57)

        
        img0 = PhotoImage(file = f'.\\image\\' + self.fname[self.im_cup[self.round][self.i]])
        
        b0 = Button(
            image = img0,
            borderwidth = 0,
            highlightthickness = 0,
            command = self.left_room_click,
            relief = "flat")

        b0.place(
            x = 6, y = 103,
            width = 500,
            height = 400)
    
        
        img1 = PhotoImage(file = f'.\\image\\' + self.fname[self.im_cup[self.round][self.i+1]])
        
        b1 = Button(
            image = img1,
            borderwidth = 0,
            highlightthickness = 0,
            command = self.right_room_click,
            relief = "flat")

        b1.place(
            x = 534, y = 103,
            width = 500,
            height = 400)
        
        window.resizable(False, False)
        window.mainloop()
        
    
    def left_room_click(self):
        left_room = self.c_cup[self.round][self.i]
        left_image = self.im_cup[self.round][self.i]
        left_room_condition = self.rc_cup[self.round][self.i]
        
        right_room_condition = self.rc_cup[self.round][self.i+1] #선택되지 않은 오른쪽 방 조건 (출력용)
        right_room = self.c_cup[self.round][self.i+1] # (dictionary)
        
        # 결승에서 왼쪽방을 선택한 경우 
        if int(self.round) == 2:
            self.c_cup[int(self.round/2)] = left_room
            self.im_cup[int(self.round/2)] = left_image
            self.rc_cup[int(self.round/2)] = left_room_condition
            
            self.room_final["월세"] = left_room["월세"]*(0.7) + right_room["월세"]*(0.3)
            self.room_final["보증금"] = left_room["보증금"]*(0.7) + right_room["보증금"]*(0.3)
            self.room_final["전세"] = left_room["전세"]*(0.7) + right_room["전세"]*(0.3)
            self.room_final["방 수"] = left_room["방 수"]*(0.7) + right_room["방 수"]*(0.3)
            self.room_final["욕실 수"] = left_room["욕실 수"]*(0.7) + right_room["욕실 수"]*(0.3)
            self.room_final["평수"] = left_room["평수"]*(0.7) + right_room["평수"]*(0.3)
            self.room_final["관리비"] = left_room["관리비"]*(0.7) + right_room["관리비"]*(0.3)
            self.room_final["주차여부"] = left_room["주차여부"]*(0.7) + right_room["주차여부"]*(0.3)
            self.room_final["층"] = left_room["층"]*(0.7) + right_room["층"]*(0.3)
            self.room_final["총 층"] = left_room["총 층"]*(0.7) + right_room["총 층"]*(0.3)
            self.room_final["복층 여부"] = left_room["복층 여부"]*(0.7) + right_room["복층 여부"]*(0.3)
            self.room_final["사용승인일"] = left_room["사용승인일"]*(0.7) + right_room["사용승인일"]*(0.3)
            self.room_final["대형마트"] = left_room["대형마트"]*(0.7) + right_room["대형마트"]*(0.3)
            self.room_final["편의점"] = left_room["편의점"]*(0.7) + right_room["편의점"]*(0.3)
            self.room_final["지하철"] = left_room["지하철"]*(0.7) + right_room["지하철"]*(0.3)
            self.room_final["방향_동"] = left_room["방향_동"]*(0.7) + right_room["방향_동"]*(0.3)
            self.room_final["방향_서"] = left_room["방향_서"]*(0.7) + right_room["방향_서"]*(0.3)
            self.room_final["방향_남"] = left_room["방향_남"]*(0.7) + right_room["방향_남"]*(0.3)
            self.room_final["방향_북"] = left_room["방향_북"]*(0.7) + right_room["방향_북"]*(0.3)
            
            self.win_room = self.room_final
            
        
        elif int(self.round)==4:
            
            room_4 = {'보증금': 0, '월세': 0, '전세': 0, '방 수': 1.38, '욕실 수': 1, '주소': 'none', '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
            
            room_4["월세"] = left_room["월세"]*(0.7) + right_room["월세"]*(0.3)
            room_4["보증금"] = left_room["보증금"]*(0.7) + right_room["보증금"]*(0.3)
            room_4["전세"] = left_room["전세"]*(0.7) + right_room["전세"]*(0.3)
            room_4["방 수"] = left_room["방 수"]*(0.7) + right_room["방 수"]*(0.3)
            room_4["욕실 수"] = left_room["욕실 수"]*(0.7) + right_room["욕실 수"]*(0.3)
            room_4["평수"] = left_room["평수"]*(0.7) + right_room["평수"]*(0.3)
            room_4["관리비"] = left_room["관리비"]*(0.7) + right_room["관리비"]*(0.3)
            room_4["주차여부"] = left_room["주차여부"]*(0.7) + right_room["주차여부"]*(0.3)
            room_4["층"] = left_room["층"]*(0.7) + right_room["층"]*(0.3)
            room_4["총 층"] = left_room["총 층"]*(0.7) + right_room["총 층"]*(0.3)
            room_4["복층 여부"] = left_room["복층 여부"]*(0.7) + right_room["복층 여부"]*(0.3)
            room_4["사용승인일"] = left_room["사용승인일"]*(0.7) + right_room["사용승인일"]*(0.3)
            room_4["대형마트"] = left_room["대형마트"]*(0.7) + right_room["대형마트"]*(0.3)
            room_4["편의점"] = left_room["편의점"]*(0.7) + right_room["편의점"]*(0.3)
            room_4["지하철"] = left_room["지하철"]*(0.7) + right_room["지하철"]*(0.3)
            room_4["방향_동"] = left_room["방향_동"]*(0.7) + right_room["방향_동"]*(0.3)
            room_4["방향_서"] = left_room["방향_서"]*(0.7) + right_room["방향_서"]*(0.3)
            room_4["방향_남"] = left_room["방향_남"]*(0.7) + right_room["방향_남"]*(0.3)
            room_4["방향_북"] = left_room["방향_북"]*(0.7) + right_room["방향_북"]*(0.3)
            
            self.c_cup[int(self.round/2)].append(room_4)
            self.im_cup[int(self.round/2)].append(left_image)
            self.rc_cup[int(self.round/2)].append(left_room_condition)
            
        
        else:
            self.c_cup[int(self.round/2)].append(left_room)
            self.im_cup[int(self.round/2)].append(left_image)
            self.rc_cup[int(self.round/2)].append(left_room_condition)
        
        if int(self.round) > 4: # 4강 이하일 때 탈락한 매물들만 csm12에 추가.
            self.csm12.append(right_room)
            self.csm12_condition.append(right_room_condition)
        
        if int((self.i+2)/2) != int(self.round/2):
            self.i += 2
            
            label = Label(window, text = "{}강 {}/{}".format(int(self.round), int((self.i+2)/2), int(self.round/2)),
                        background = "white", font = ("Times", 32))
            label.place(x = 0, y = 8, width = 200, height = 65)

            room1_condition = Label(window, text = "{}".format(self.show_room_condition(self.round, self.i)),
                            background = "white", font = ("Times", 16, "bold"))
            room1_condition.place(x = 10, y = 538, width = 500, height = 57)

            room2_condition = Label(window, text = "{}".format(self.show_room_condition(self.round, self.i+1)),
                            background = "white", font = ("Times", 16, "bold"))
            room2_condition.place(x = 520, y = 538, width = 500, height = 57)

            img0 = PhotoImage(file = f'.\\image\\' + self.fname[self.im_cup[self.round][self.i]])
            b0 = Button(
                image = img0,
                borderwidth = 0,
                highlightthickness = 0,
                command = self.left_room_click,
                relief = "flat")

            b0.place(
                x = 6, y = 103,
                width = 500,
                height = 400)

            img1 = PhotoImage(file = f'.\\image\\' + self.fname[self.im_cup[self.round][self.i+1]])
            b1 = Button(
                image = img1,
                borderwidth = 0,
                highlightthickness = 0,
                command = self.right_room_click,
                relief = "flat")

            b1.place(
                x = 534, y = 103,
                width = 500,
                height = 400)
            
            window.resizable(False, False)
            window.mainloop()
            
        else:
            self.i = 0
            self.round = int(self.round / 2)
            if int(self.round) == 1:
                self.victory()
            else:
                self.start_game()
            
    def right_room_click(self):
        right_room = self.c_cup[self.round][self.i+1]
        right_image = self.im_cup[self.round][self.i+1]
        right_room_condition = self.rc_cup[self.round][self.i+1]
        
        left_room_condition = self.rc_cup[self.round][self.i] #선택되지 않은 왼쪽 방 조건 (출력용)
        left_room = self.c_cup[self.round][self.i] # (dictionary)
        
        if int(self.round) == 2:
            self.c_cup[int(self.round/2)] = right_room
            self.im_cup[int(self.round/2)] = right_image
            self.rc_cup[int(self.round/2)] = right_room_condition
            
            self.room_final["월세"] = left_room["월세"]*(0.3) + right_room["월세"]*(0.7)
            self.room_final["보증금"] = left_room["보증금"]*(0.3) + right_room["보증금"]*(0.7)
            self.room_final["전세"] = left_room["전세"]*(0.3) + right_room["전세"]*(0.7)
            self.room_final["방 수"] = left_room["방 수"]*(0.3) + right_room["방 수"]*(0.7)
            self.room_final["욕실 수"] = left_room["욕실 수"]*(0.3) + right_room["욕실 수"]*(0.7)
            self.room_final["평수"] = left_room["평수"]*(0.3) + right_room["평수"]*(0.7)
            self.room_final["관리비"] = left_room["관리비"]*(0.3) + right_room["관리비"]*(0.7)
            self.room_final["주차여부"] = left_room["주차여부"]*(0.3) + right_room["주차여부"]*(0.7)
            self.room_final["층"] = left_room["층"]*(0.3) + right_room["층"]*(0.7)
            self.room_final["총 층"] = left_room["총 층"]*(0.3) + right_room["총 층"]*(0.7)
            self.room_final["복층 여부"] = left_room["복층 여부"]*(0.3) + right_room["복층 여부"]*(0.7)
            self.room_final["사용승인일"] = left_room["사용승인일"]*(0.3) + right_room["사용승인일"]*(0.7)
            self.room_final["대형마트"] = left_room["대형마트"]*(0.3) + right_room["대형마트"]*(0.7)
            self.room_final["편의점"] = left_room["편의점"]*(0.3) + right_room["편의점"]*(0.7)
            self.room_final["지하철"] = left_room["지하철"]*(0.3) + right_room["지하철"]*(0.7)
            self.room_final["방향_동"] = left_room["방향_동"]*(0.3) + right_room["방향_동"]*(0.7)
            self.room_final["방향_서"] = left_room["방향_서"]*(0.3) + right_room["방향_서"]*(0.7)
            self.room_final["방향_남"] = left_room["방향_남"]*(0.3) + right_room["방향_남"]*(0.7)
            self.room_final["방향_북"] = left_room["방향_북"]*(0.3) + right_room["방향_북"]*(0.7)
            
            self.win_room = self.room_final
            print(self.win_room)
            
        elif int(self.round)==4:
            room_4 = {'보증금': 0, '월세': 0, '전세': 0, '방 수': 1.38, '욕실 수': 1, '주소': 'none', '평수': 8.62,
            '관리비': 5.5, '주차여부': 0.45, '층': 0.5, '총 층': 3.9, '복층 여부': 0.2, '사용승인일': 1984,
            '대형마트': 447, '편의점': 83, '지하철': 411, '방향_동': 0, '방향_서': 0, '방향_남': 0, '방향_북': 0}
            
            room_4["월세"] = left_room["월세"]*(0.3) + right_room["월세"]*(0.7)
            room_4["보증금"] = left_room["보증금"]*(0.3) + right_room["보증금"]*(0.7)
            room_4["전세"] = left_room["전세"]*(0.3) + right_room["전세"]*(0.7)
            room_4["방 수"] = left_room["방 수"]*(0.3) + right_room["방 수"]*(0.7)
            room_4["욕실 수"] = left_room["욕실 수"]*(0.3) + right_room["욕실 수"]*(0.7)
            room_4["평수"] = left_room["평수"]*(0.3) + right_room["평수"]*(0.7)
            room_4["관리비"] = left_room["관리비"]*(0.3) + right_room["관리비"]*(0.7)
            room_4["주차여부"] = left_room["주차여부"]*(0.3) + right_room["주차여부"]*(0.7)
            room_4["층"] = left_room["층"]*(0.3) + right_room["층"]*(0.7)
            room_4["총 층"] = left_room["총 층"]*(0.3) + right_room["총 층"]*(0.7)
            room_4["복층 여부"] = left_room["복층 여부"]*(0.3) + right_room["복층 여부"]*(0.7)
            room_4["사용승인일"] = left_room["사용승인일"]*(0.3) + right_room["사용승인일"]*(0.7)
            room_4["대형마트"] = left_room["대형마트"]*(0.3) + right_room["대형마트"]*(0.7)
            room_4["편의점"] = left_room["편의점"]*(0.3) + right_room["편의점"]*(0.7)
            room_4["지하철"] = left_room["지하철"]*(0.3) + right_room["지하철"]*(0.7)
            room_4["방향_동"] = left_room["방향_동"]*(0.3) + right_room["방향_동"]*(0.7)
            room_4["방향_서"] = left_room["방향_서"]*(0.3) + right_room["방향_서"]*(0.7)
            room_4["방향_남"] = left_room["방향_남"]*(0.3) + right_room["방향_남"]*(0.7)
            room_4["방향_북"] = left_room["방향_북"]*(0.3) + right_room["방향_북"]*(0.7)
            
            self.c_cup[int(self.round/2)].append(room_4)
            self.im_cup[int(self.round/2)].append(right_image)
            self.rc_cup[int(self.round/2)].append(right_room_condition)
            
        else:
            self.c_cup[int(self.round/2)].append(right_room)
            self.im_cup[int(self.round/2)].append(right_image)
            self.rc_cup[int(self.round/2)].append(right_room_condition)
        
        if int(self.round) > 4: # 4강 이하일 때 탈락한 매물들만 csm12에 추가.
            self.csm12.append(left_room)
            self.csm12_condition.append(left_room_condition)
            
        if int((self.i+2)/2) != int(self.round/2):
            self.i += 2
            
            label = Label(window, text = "{}강 {}/{}".format(int(self.round), int((self.i+2)/2), int(self.round/2)),
                        background = "white", font = ("Times", 32))
            label.place(x = 0, y = 8, width = 200, height = 65)

            room1_condition = Label(window, text = "{}".format(self.show_room_condition(self.round, self.i)),
                            background = "white", font = ("Times", 16, "bold"))
            room1_condition.place(x = 10, y = 538, width = 500, height = 57)

            room2_condition = Label(window, text = "{}".format(self.show_room_condition(self.round, self.i+1)),
                            background = "white", font = ("Times", 16, "bold"))
            room2_condition.place(x = 520, y = 538, width = 500, height = 57)

            img0 = PhotoImage(file = f'.\\image\\' + self.fname[self.im_cup[self.round][self.i]])
            b0 = Button(
                image = img0,
                borderwidth = 0,
                highlightthickness = 0,
                command = self.left_room_click,
                relief = "flat")

            b0.place(
                x = 6, y = 103,
                width = 500,
                height = 400)

            img1 = PhotoImage(file = f'.\\image\\' + self.fname[self.im_cup[self.round][self.i+1]])
            b1 = Button(
                image = img1,
                borderwidth = 0,
                highlightthickness = 0,
                command = self.right_room_click,
                relief = "flat")

            b1.place(
                x = 534, y = 103,
                width = 500,
                height = 400)
            
            window.resizable(False, False)
            window.mainloop()
            
        else:
            self.i = 0
            self.round = int(self.round / 2)
            if int(self.round) == 1:
                self.victory()
            else:
                self.start_game()
            

## 추천 시스템

In [12]:
class RecommendSystem:
    
    def __init__(self, district,neighborhood,rent,user):
        data_name = "data_" + district +"_"+ neighborhood
        print(data_name)
        self.data = pd.read_csv("C:/Users/user/Desktop/이상형월드컵/"+str(data_name)+"_preprocessed.csv",encoding="utf-8-sig")
        self.rent = rent
        self.rent_max = user['월세']
        self.deposit_max = user['보증금']
        self.jeonse_max = user['전세']
        i = 1
        arr = []
        for key in user:
            if i > 3:
                arr.append(user[key])
            i += 1
        self.userItem = [tuple(arr)]
        self.top_n = 7
    
    def recommend(self):
        # 가격 범위 밖의 데이터 삭제
        if self.rent == '월세':
            if (self.rent_max != 0) | (self.deposit_max != 0):
                if (self.rent_max != 0) & (self.deposit_max != 0):
                    condition = (self.rent_max >= self.data['월세']) & (self.deposit_max >= self.data['보증금'])
                elif self.rent_max != 0:
                    condition = (self.rent_max >= self.data['월세'])
                else:
                    condition = (self.deposit_max >= self.data['보증금'])
                self.data = self.data[condition]
        else:
            if self.jeonse_max != 0:
                condition = (self.data['전세'] <= self.jeonse_max)
                self.data = self.data[condition]

        # 추천 알고리즘에 사용하지 않는 열 삭제
        col_list = ['방 수', '욕실 수', '평 수', '관리비', '주차 여부', '층', '총 층', '복층 여부', '사용승인일', '대형마트', '편의점', '지하철역', '방향_동', '방향_서', '방향_남', '방향_북']
        df = self.data[col_list]
        
        # 유저 아이템 생성
        u_df = pd.DataFrame(self.userItem, columns=df.columns, index=['user'])
        df = df.append(u_df)
        
        # 정규화 
        min_df = df.min()
        max_df = df.max()
        normalization_df = (df - min_df) / (max_df-min_df)
        if min_df['욕실 수'] == max_df['욕실 수']:
            normalization_df['욕실 수'] = df['욕실 수']
        
        # 코사인 유사도 계산
        result = df.copy()
        
        user = normalization_df.loc['user'].values
        for idx in df.index:
            item = normalization_df.loc[idx].values
            similarity = 1 - spatial.distance.cosine(item, user)
            result.loc[idx, '유사도'] = similarity
        
        # 결과에서 유저 데이터 삭제, 유사도 컬럼만 추출
        result = result.drop(['user'], axis = 0)
#         result = result['유사도']
        
        # 결과 중 상위 n개 추출 
        result = result['유사도'].sort_values(ascending=False)
        result = result.head(self.top_n)
#         print(result)
        
        # 최종 추천 아이템들의 기존 컬럼 조인
        result_data = self.data.loc[result.index]
#         print(result_data)
#         recommend = result_data.join(result_top, how='left')
        
        return result_data.to_dict('records')

## 메일 코드

In [7]:
class mailer:
    
    def __init__(self):
        self.text = ""
    
    # list로 넘겨주세요, rent 는 월세 여부 
    def get_room_list(self,rent,room_list):
        if (rent == "월세"):
            for i in room_list:
                room_name = "방이름: "+i["방이름"] + '\n'
                bo = "보증금: "+ str(i["보증금"]) +"만원"+ '\n'
                rent_money = "월세: "+str(i["월세"])+"만원"+'\n'
                room_num = "방 수: "+str(i["방 수"])+"개"'\n'
                bath_num = "욕실 수: "+str(i["욕실 수"])+"개" + '\n'
                address = "주소: " +i["주소"] + '\n'
                pyung = "평수: " + str(i["평 수"])+"평" + '\n'
                admin = "관리비: "+str(int(i["관리비"]*10000))+"원" + '\n'
                
                if(i["주차 여부"]==1):
                    park = "주차 가능" + '\n'
                else:
                    park = "주차 불가능" + '\n'
                    
                floor = "층/총 층" + str(i["층"]) + "/" + str(i["총 층"]) + '\n'
                
                if(i["복층 여부"]==1):
                    two = "복층" +'\n'
                else:
                    two = "복층X" + '\n'
                
                year = "사용승인일: " + str(i["사용승인일"]) + "년"+"\n"
                
                if(i["내부시설"]!="none"):
                    etc1 = "내부시설: " + i["내부시설"] + '\n'
                else:
                    etc1 = "내부시설 정보 없음 \n"
                
                if(i["기타시설"]!= "none"):
                    etc2 = "기타시설" + i["기타시설"]+'\n'
                etc2 = "기타시설 정보 없음 \n"
                
                
                real_est = "\n부동산 이름: " + i["부동산 이름"] +'\n'
                real_est_phone = "부동산 전화번호: " + i["부동산 전화번호"]+'\n'
                
                blank = "--------------------------------------------------------------------\n"
                
                self.text += (room_name + bo + rent_money+room_num+bath_num+address+pyung+admin+park+floor+two+year+etc1+etc2+real_est+real_est_phone+blank)
            
                
        else:
            for i in (room_list):
                room_name = "방이름: "+i["방이름"] + '\n'
                rent_money = "전세: "+str(i["전세"])+"만원"+'\n'
                room_num = "방 수: "+str(i["방 수"])+"개"'\n'
                bath_num = "욕실 수: "+str(i["욕실 수"])+"개" + '\n'
                address = "주소: " +i["주소"] + '\n'
                pyung = "평수: " + str(i["평 수"])+"평" + '\n'
                admin = "관리비: "+str(int(i["관리비"]*10000))+"원" + '\n'
                
                if(i["주차 여부"]==1):
                    park = "주차 가능" + '\n'
                else:
                    park = "주차 불가능" + '\n'
                    
                floor = "층/총 층" + str(i["층"]) + "/" + str(i["총 층"]) + '\n'
                
                if(i["복층 여부"]==1):
                    two = "복층" +'\n'
                else:
                    two = "복층X" + '\n'
                
                year = "사용승인일: " + str(i["사용승인일"]) + "년"+"\n"
                
                if(i["내부시설"]!="none"):
                    etc1 = "내부시설: " + i["내부시설"] + '\n'
                else:
                    etc1 = "내부시설 정보 없음 \n"
                
                if(i["기타시설"]!= "none"):
                    etc2 = "기타시설" + i["기타시설"]+'\n'
                etc2 = "기타시설 정보 없음 \n"
                
                
                real_est = "\n부동산 이름: " + i["부동산 이름"] +'\n'
                real_est_phone = "부동산 전화번호: " + i["부동산 전화번호"]+'\n'
                
                blank = "--------------------------------------------------------------------\n"
                self.text += (room_name + rent_money+room_num+bath_num+address+pyung+admin+park+floor+two+year+etc1+etc2+real_est+real_est_phone+blank)
        print(self.text)
    
    def send_email(self,email):
        sendEmail = "uiseok0227@naver.com" # 보내는 사람 이메일
        recvEmail = email  # 사용자의 이메일(앞의 GUI에서 받음)
        password = #비밀번호

        smtpName = "smtp.naver.com" #smtp 서버 주소
        smtpPort = 587 #smtp 포트 번호

        text = '''
        [이상형 월드컵을 통한 서울시 자취방 추천] 결과입니다. 
        아래 7개의 방이 추천되었습니다.
    
        '''
        text+=self.text
        
        ending = '''
        
        이용해주셔서 감사합니다!
        '''
        text += ending
        print(text)
        
        msg = MIMEText(text) #MIMEText(text , _charset = "utf8")

        msg['Subject'] ="[이상형 월드컵을 통한 서울시 자취방 추천]"
        msg['From'] = sendEmail
        msg['To'] = recvEmail
        print(msg.as_string())

        s=smtplib.SMTP( smtpName , smtpPort ) #메일 서버 연결
        s.starttls() #TLS 보안 처리
        s.login( sendEmail , password ) #로그인
        s.sendmail( sendEmail, recvEmail, msg.as_string() ) #메일 전송, 문자열로 변환하여 보냅니다.
        s.close() #smtp 서버 연결을 종료합니다.

## 실행

In [17]:
w = WorldCup()
window = Tk()

rent = StringVar()
rent_fee = IntVar()
deposit = IntVar()
district = StringVar()
neighborhood = StringVar()
email = StringVar()

def btn_exit():
    window.destroy()

def start_gui():
    window.title("자취방 이상형월드컵")
    window.geometry("1040x616+270+90")
    window.configure(bg = "#ffffff")
    canvas = Canvas(
        window,
        bg = "#ffffff",
        height = 616,
        width = 1040,
        bd = 0,
        highlightthickness = 0,
        relief = "ridge")

    canvas.place(x = 0, y = 0)
    
    btn_rent1 = Radiobutton(window, text= "월세", relief = "groove", value = "월세", variable = rent)
    btn_rent2 = Radiobutton(window, text= "전세", relief = "groove", value = "전세", variable = rent)
    btn_rent1.select()
    btn_rent1.place(x=640, y=73)
    btn_rent2.place(x=640, y=95)
    
    textbox0_img = PhotoImage(file = f"img_textBox0.png")
    textbox0_bg = canvas.create_image(
        608.5, 149.0,
        image = textbox0_img)

    textbox0 = Entry(
        bd = 0,
        bg = "#e9e9e9",
        highlightthickness = 0,
        textvariable = rent_fee)

    textbox0.place(
        x = 548.0, y = 119,
        width = 121.0,
        height = 58)

    textbox1_img = PhotoImage(file = f"img_textBox1.png")
    textbox1_bg = canvas.create_image(
        830.5, 149.0,
        image = textbox1_img)

    textbox1 = Entry(
        bd = 0,
        bg = "#e9e9e9",
        highlightthickness = 0,
        textvariable = deposit)

    textbox1.place(
        x = 770.0, y = 119,
        width = 121.0,
        height = 58)

    textbox2_img = PhotoImage(file = f"img_textBox2.png")
    textbox2_bg = canvas.create_image(
        719.5, 242.0,
        image = textbox2_img)

    textbox2 = Entry(
        bd = 0,
        bg = "#e9e9e9",
        highlightthickness = 0,
        textvariable = district)

    textbox2.place(
        x = 548.0, y = 212,
        width = 343.0,
        height = 58)

    textbox3_img = PhotoImage(file = f"img_textBox3.png")
    textbox3_bg = canvas.create_image(
        719.5, 335.0,
        image = textbox3_img)

    textbox3 = Entry(
        bd = 0,
        bg = "#e9e9e9",
        highlightthickness = 0,
        textvariable = neighborhood)

    textbox3.place(
        x = 548.0, y = 305,
        width = 343.0,
        height = 58)
    
    textbox4_img = PhotoImage(file = f"img_textBox4.png")
    textbox4_bg = canvas.create_image(
        719.5, 428.0,
        image = textbox4_img)

    textbox4 = Entry(
        bd = 0,
        bg = "#e9e9e9",
        highlightthickness = 0,
        textvariable = email)

    textbox4.place(
        x = 548.0, y = 398,
        width = 343.0,
        height = 58)

    img0 = PhotoImage(file = f"start.png")
    b0 = Button(
        image = img0,
        borderwidth = 0,
        highlightthickness = 0,
        command = w.start_game,
        relief = "flat")

    b0.place(
        x = 603, y = 475,
        width = 240,
        height = 102)
    
    background_img = PhotoImage(file = f"background.png")
    background = canvas.create_image(
        502.5, 236.0,
        image=background_img)

    window.resizable(False, False)
    window.mainloop()

start_gui()
print("-------우승 매물-------")
print(w.get_room())
print("-------이상형 월드컵 종료-------")
# 크롤링 시작     
a = croll()
a.input_data(w.get_rent(),w.get_district(),w.get_neighborhood(),w.get_email())
a.crolling()
print("--------크롤링 종료---------")
time.sleep(1)

# 데이터 전처리 
b = preprocess(w.get_district(),w.get_neighborhood())
b.process()
print("--------전처리 종료---------")
# 추천 알고리즘
c = RecommendSystem(w.get_district(),w.get_neighborhood(), w.get_rent(), w.get_room())
# room = {'보증금': 466.40000000000003, '월세': 47.48, '전세': 0.0, '방 수': 1.38, '욕실 수': 1.0, '평수': 8.16896, '관리비': 5.5, '주차여부': 0.4248, '층': 0.752, '총 층': 3.9000000000000004, '복층 여부': 0.18, '사용승인일': 1984.224, '대형마트': 461.544, '편의점': 83.0, '지하철': 366.2, '방향_동': 0.0, '방향_서': 0.0, '방향_남': 0.072, '방향_북': 0.0}
# c = RecommendSystem("강북구","우이동","월세",w.get_room())
room_list = c.recommend()
print("--------추천 알고리즘 종료---------")
print(room_list)
# # 메일
d = mailer()
# mail.get_room_list(w.get_rent(),room_list)
# a = "월세"
d.get_room_list(w.get_rent(),room_list)
d.send_email(w.get_email())

-------우승 매물-------
{'보증금': 500.0, '월세': 50.0, '전세': 0.0, '방 수': 1.4203199999999998, '욕실 수': 1.0, '평수': 9.95632, '관리비': 6.9, '주차여부': 0.38000000000000006, '층': 1.424, '총 층': 3.9000000000000004, '복층 여부': 0.1928, '사용승인일': 1988.3680000000002, '대형마트': 416.0, '편의점': 86.696, '지하철': 411.0, '방향_동': 0.0, '방향_서': 0.0, '방향_남': 0.2, '방향_북': 0.0}
-------이상형 월드컵 종료-------
--------크롤링 종료---------
--------전처리 종료---------
data_강북구_우이동
--------추천 알고리즘 종료---------
[{'방이름': '일반원룸2층', '보증금': 500, '월세': 30, '전세': 0, '방 수': 2, '욕실 수': 1, '주소': '서울시 강북구 우이동 72-157', '평 수': 9.07, '관리비': 5.0, '주차 여부': 0, '층': 4, '총 층': 5, '전세여부': 0, '월세여부': 1, '부동산 이름': '신하나공인중개사사무소', '부동산 전화번호': '02-996-1134,010-6342-4597', '복층 여부': 0, '사용승인일': 2019, '내부시설': '벽걸이에어컨, ', '기타시설': 'none', '방향_동': 1.0, '방향_서': 0.0, '방향_남': 0.0, '방향_북': 0.0, '건축물 용도_공동주택': 0, '건축물 용도_단독주택': 1, '건축물 용도_제2종 근린생활시설': 0, '주소좌표': "['127.01295088499', '37.6501973162387']", '대형마트': 703, '편의점': 92, '지하철역': 99}, {'방이름': '단독2층', '보증금': 500, '월세': 30, '전세': 0,